# 1. 순환 신경망 RNN

## Numpy로 순환 신경망 구현하기

In [ ]:
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_dim = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_dim)) # 입력에 해당되는 2D 텐서

# 은닉 상태의 크기 hidden_size로 은닉 상태를 만든다.
hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화

In [ ]:
print(hidden_state_t) # 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐.

[0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
Wx = np.random.random((hidden_size, input_dim))  # (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
b = np.random.random((hidden_size,)) # (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).

In [ ]:
print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [ ]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨.
  output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) # Wx * Xt + Wh * Ht-1 + b(bias)
  total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적
  print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
  hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0) 
# 출력 시 값을 깔끔하게 해준다.

print(total_hidden_states) # (timesteps, output_dim)의 크기. 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.91391596 0.91198046 0.973877   0.41717514 0.90204106 0.90876622
  0.8869713  0.85048275]
 [0.99999893 0.99998373 0.99998539 0.99922499 0.99995356 0.99974347
  0.99976692 0.99948329]
 [0.99999967 0.99998751 0.99999478 0.99988551 0.99999334 0.99990435
  0.99993471 0.99956022]
 [0.9999998  0.99999151 0.99999463 0.99987614 0.99999498 0.9999452
  0.99993689 0.99974377]
 [0.99999943 0.99999042 0.99999444 0.99979308 0.99997669 0.99987004
  0.99987005 0.99972147]
 [0.99999976 0.99999084 0.9999963  0.99988601 0.99999445 0.99993322
  0.99994933 0.99973169]
 [0.99999959 0.99998847 0.99999123 0.99980814 0.9999892  0.99980489
  0.99989285 0.99941045]
 [0.99999979 0.99999094 0.99999586 0.99988305 0.99999582 0.99992009
  0.99995453 0.99966936]
 [0.9999999  0.99999636 0.99999762 0.99984534 0.99999629 0.9999648
  0.99996473 0.99991356]
 [0.99999968 0.99999062 0.99999127 0.99982809 0.99998916 0.99992403
  0.99987553 0.99970603]]


## 순환 신경망 구현 이해하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Bidirectional, LSTM, GRU

In [ ]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2, 10)))
# model.add(SimpleRNN(3, input_length=2, input_dim=10))와 동일.
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


input_length = time_step

파라미터수가 왜 42가 되는지 생각. 입력 차원이 10일 때, hidden_size는 3.  
참고로 time_step은 파라미터 수에 전혀 영향을 주지 않음.  

출력값이 (batch_size, output_dim) 크기의 2D 텐서일 때, output_dim은 hidden_size의 값인 3. 이 경우 batch_size를 현 단계에서는 알 수 없으므로 (None, 3)이 됨. 이번에는 batch_size를 미리 정의.

In [ ]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(10,10), return_sequences=True))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 10, 3)             42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


배치 크기는 추정이 안 되므로 None이고, 문장의 길이. 즉, timesteps는 10이므로 (None, 10, 3)입니다.

In [ ]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (8, 3)                    42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


batch_size를 8로 기재하자, 출력의 크기가 (8, 3)이 된 것을 볼 수 있음. 이제 return_sequences 매개 변수에 True를 기재하여 출력값으로 (batch_size, timesteps, output_dim) 크기의 3D 텐서를 리턴하도록 모델을 생성.

In [ ]:
model = Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


출력의 크기가 (8, 2, 3)이 된 것을 확인할 수 있음.

## 은닉층이 2개인 깊은 순환 신경망

In [ ]:
model = Sequential()
model.add(SimpleRNN(hidden_size, return_sequences = True))
model.add(SimpleRNN(hidden_size, return_sequences = True))

## 양방향 순환 신경망

In [ ]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))

## 깊은 양방향 순환 신경망

In [ ]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences = True)))

## LSTM 구현하기

In [ ]:
# 실제 LSTM 은닉층을 추가하는 코드.
model = Sequential()
model.add(LSTM(hidden_size, input_shape=(timesteps, input_dim)))

## GRU 구현하기

In [ ]:
# 실제 GRU 은닉층을 추가하는 코드.
model = Sequential()
model.add(GRU(hidden_size, input_shape=(timesteps, input_dim)))

# RNN 언어 모델 (char 단위)

## 데이터 전처리

In [ ]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

In [ ]:
urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")
f = open('11-0.txt', 'rb')
lines=[]
for line in f: # 데이터를 한 줄씩 읽는다.
    line=line.strip() # strip()을 통해 \r, \n을 제거한다.
    line=line.lower() # 소문자화.
    line=line.decode('ascii', 'ignore') # \xe2\x80\x99 등과 같은 바이트 열 제거
    if len(line) > 0:
        lines.append(line)
f.close()

In [ ]:
lines[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  you may copy it, give it away or',
 're-use it under the terms of the project gutenberg license included',
 'with this ebook or online at www.gutenberg.org']

In [ ]:
text = ' '.join(lines)
print('문자열의 길이 또는 총 글자의 개수: %d' % len(text))

문자열의 길이 또는 총 글자의 개수: 159612


In [ ]:
print(text[:200])

the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  you may copy it, g


In [ ]:
char_vocab = sorted(list(set(text)))
vocab_size=len(char_vocab)
print ('글자 집합의 크기 : {}'.format(vocab_size))

글자 집합의 크기 : 57


In [ ]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print(char_to_index)

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '@': 27, '[': 28, ']': 29, '_': 30, 'a': 31, 'b': 32, 'c': 33, 'd': 34, 'e': 35, 'f': 36, 'g': 37, 'h': 38, 'i': 39, 'j': 40, 'k': 41, 'l': 42, 'm': 43, 'n': 44, 'o': 45, 'p': 46, 'q': 47, 'r': 48, 's': 49, 't': 50, 'u': 51, 'v': 52, 'w': 53, 'x': 54, 'y': 55, 'z': 56}


In [ ]:
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key

Example) 샘플의 길이가 4라면 4개의 입력 글자 시퀀스로 부터 4개의 출력 글자  시퀀스를 예측.  
즉, RNN의 time step은 4번  
appl -> pple  
appl은 train_X(입력 시퀀스), pple는 train_y(예측해야하는 시퀀스)에 저장한다.

In [ ]:
seq_length = 60 # 문장의 길이를 60으로 한다.
n_samples = int(np.floor((len(text) - 1) / seq_length)) # 문자열을 60등분한다. 그러면 즉, 총 샘플의 개수
print ('문장 샘플의 수 : {}'.format(n_samples))

문장 샘플의 수 : 2660


In [ ]:
train_X = []
train_y = []

for i in range(n_samples): # 2,646번 수행
    X_sample = text[i * seq_length: (i + 1) * seq_length]
    # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 가져온다.
    X_encoded = [char_to_index[c] for c in X_sample] # 하나의 문장 샘플에 대해서 정수 인코딩
    train_X.append(X_encoded)

    y_sample = text[i * seq_length + 1: (i + 1) * seq_length + 1] # 오른쪽으로 1칸 쉬프트한다.
    y_encoded = [char_to_index[c] for c in y_sample]
    train_y.append(y_encoded)

In [ ]:
print(train_X[0])

[50, 38, 35, 0, 46, 48, 45, 40, 35, 33, 50, 0, 37, 51, 50, 35, 44, 32, 35, 48, 37, 0, 35, 32, 45, 45, 41, 0, 45, 36, 0, 31, 42, 39, 33, 35, 49, 0, 31, 34, 52, 35, 44, 50, 51, 48, 35, 49, 0, 39, 44, 0, 53, 45, 44, 34, 35, 48, 42, 31]


In [ ]:
print(train_y[0])

[38, 35, 0, 46, 48, 45, 40, 35, 33, 50, 0, 37, 51, 50, 35, 44, 32, 35, 48, 37, 0, 35, 32, 45, 45, 41, 0, 45, 36, 0, 31, 42, 39, 33, 35, 49, 0, 31, 34, 52, 35, 44, 50, 51, 48, 35, 49, 0, 39, 44, 0, 53, 45, 44, 34, 35, 48, 42, 31, 44]


In [ ]:
print(train_X[1])

[44, 34, 10, 0, 32, 55, 0, 42, 35, 53, 39, 49, 0, 33, 31, 48, 48, 45, 42, 42, 0, 50, 38, 39, 49, 0, 35, 32, 45, 45, 41, 0, 39, 49, 0, 36, 45, 48, 0, 50, 38, 35, 0, 51, 49, 35, 0, 45, 36, 0, 31, 44, 55, 45, 44, 35, 0, 31, 44, 55]


In [ ]:
print(train_y[1])

[34, 10, 0, 32, 55, 0, 42, 35, 53, 39, 49, 0, 33, 31, 48, 48, 45, 42, 42, 0, 50, 38, 39, 49, 0, 35, 32, 45, 45, 41, 0, 39, 49, 0, 36, 45, 48, 0, 50, 38, 35, 0, 51, 49, 35, 0, 45, 36, 0, 31, 44, 55, 45, 44, 35, 0, 31, 44, 55, 53]


In [ ]:
train_X = to_categorical(train_X)
train_y = to_categorical(train_y)

In [ ]:
print('train_X의 크기(shape) : {}'.format(train_X.shape)) # 원-핫 인코딩
print('train_y의 크기(shape) : {}'.format(train_y.shape)) # 원-핫 인코딩

train_X의 크기(shape) : (2660, 60, 57)
train_y의 크기(shape) : (2660, 60, 57)


이는 샘플의 수(No. of samples)가 2,646개, 입력 시퀀스의 길이(input_length)가 60, 각 벡터의 차원(input_dim)이 57임을 의미. 원-핫 벡터의 차원은 글자 집합의 크기인 57이어야 하므로 원-핫 인코딩이 수행되었음을 알 수 있음.

## 모델 설계하기

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(None, train_X.shape[2]), return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_X, train_y, epochs=80, verbose=2)

Epoch 1/80
84/84 - 1s - loss: 3.0858 - accuracy: 0.1806
Epoch 2/80
84/84 - 1s - loss: 2.7181 - accuracy: 0.2567
Epoch 3/80
84/84 - 1s - loss: 2.3902 - accuracy: 0.3287
Epoch 4/80
84/84 - 1s - loss: 2.2475 - accuracy: 0.3604
Epoch 5/80
84/84 - 1s - loss: 2.1407 - accuracy: 0.3877
Epoch 6/80
84/84 - 1s - loss: 2.0551 - accuracy: 0.4075
Epoch 7/80
84/84 - 1s - loss: 1.9821 - accuracy: 0.4268
Epoch 8/80
84/84 - 1s - loss: 1.9216 - accuracy: 0.4424
Epoch 9/80
84/84 - 1s - loss: 1.8667 - accuracy: 0.4600
Epoch 10/80
84/84 - 1s - loss: 1.8173 - accuracy: 0.4732
Epoch 11/80
84/84 - 1s - loss: 1.7725 - accuracy: 0.4839
Epoch 12/80
84/84 - 1s - loss: 1.7363 - accuracy: 0.4944
Epoch 13/80
84/84 - 1s - loss: 1.6948 - accuracy: 0.5052
Epoch 14/80
84/84 - 1s - loss: 1.6600 - accuracy: 0.5146
Epoch 15/80
84/84 - 1s - loss: 1.6235 - accuracy: 0.5238
Epoch 16/80
84/84 - 1s - loss: 1.5879 - accuracy: 0.5327
Epoch 17/80
84/84 - 1s - loss: 1.5543 - accuracy: 0.5412
Epoch 18/80
84/84 - 1s - loss: 1.5228 - 

In [ ]:
def sentence_generation(model, length):
    ix = [np.random.randint(vocab_size)] # 글자에 대한 랜덤 인덱스 생성
    y_char = [index_to_char[ix[-1]]] # 랜덤 익덱스로부터 글자 생성
    print(ix[-1],'번 글자',y_char[-1],'로 예측을 시작!')
    X = np.zeros((1, length, vocab_size)) # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성

    for i in tf.range(length):
        i = tf.cast(i, tf.int64)
        X[0][i][ix[-1]] = 1 # X[0][i][예측한 글자의 인덱스] = 1, 즉, 예측 글자를 다음 입력 시퀀스에 추가
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

In [ ]:
sentence_generation(model, 100)

35 번 글자 e 로 예측을 시작!
e to say out of the way is. station. what extled its all asked ays. as she said the last word with s

'e to say out of the way is. station. what extled its all asked ays. as she said the last word with su'